# Model Evaluation and Analysis

This notebook provides detailed evaluation and analysis of the trained models, including prediction visualization and error analysis.

## Overview
- **Purpose**: Comprehensive evaluation of trained transformer models
- **Analysis Types**: Prediction quality, error patterns, confusion matrices
- **Visualization**: Interactive prediction examples with detailed breakdowns
- **Output**: Professional evaluation reports and insights


## 1. Load Results and Setup


In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import warnings
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, multilabel_confusion_matrix, classification_report
)
from transformers import AutoTokenizer
import textwrap
from IPython.display import display, HTML

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Define label names
label_names = [
    'Regenerative & Eco-Tourism',
    'Integrated Wellness',
    'Immersive Culinary',
    'Off-the-Beaten-Path Adventure'
]

# Load training results
try:
    with open('training_results.json', 'r') as f:
        results = json.load(f)
    print("Training results loaded successfully")
    print(f"Best model: {results['best_model']['config']['model']}")
    print(f"Best F1-Score: {results['best_model']['metrics']['f1']:.4f}")
except FileNotFoundError:
    print("Warning: training_results.json not found. Please run the training notebook first.")
    results = None

print("Evaluation setup completed")


## 2. Prediction Visualization Functions


In [ ]:
def analyze_prediction_quality(y_true, y_pred, label_names, model_name="Model"):
    """Analyze prediction quality and categorize examples."""
    results = {
        'perfect_correct': [],
        'partially_correct': [],
        'completely_wrong': [],
        'false_positives': [],
        'false_negatives': [],
        'edge_cases': []
    }
    
    for i in range(len(y_true)):
        true_labels = y_true[i]
        pred_labels = y_pred[i]
        
        correct_labels = sum(pred_labels[j] == true_labels[j] for j in range(len(pred_labels)))
        total_labels = len(pred_labels)
        
        if correct_labels == total_labels:
            if sum(true_labels) > 0:
                results['perfect_correct'].append(i)
        elif correct_labels == 0:
            results['completely_wrong'].append(i)
        else:
            results['partially_correct'].append(i)
        
        for j in range(len(pred_labels)):
            if pred_labels[j] == 1 and true_labels[j] == 0:
                results['false_positives'].append((i, j, label_names[j]))
            elif pred_labels[j] == 0 and true_labels[j] == 1:
                results['false_negatives'].append((i, j, label_names[j]))
        
        if sum(pred_labels) == 0 and sum(true_labels) > 0:
            results['edge_cases'].append((i, "Missed all positive labels"))
        elif sum(pred_labels) == total_labels and sum(true_labels) < total_labels:
            results['edge_cases'].append((i, "Over-predicted all labels"))
    
    return results

def display_prediction_example(idx, y_true, y_pred, label_names, model_name="Model", analysis_type="Example"):
    """Display a single prediction example with detailed analysis."""
    true_labels = y_true[idx]
    pred_labels = y_pred[idx]
    
    html = f"""
    <div style='background-color: #f9f9f9; padding: 15px; border-radius: 5px; margin: 10px 0;'>
        <h3 style='color: #2c3e50; margin-top: 0;'>{analysis_type} Prediction Example - {model_name}</h3>
        <p><strong>Index:</strong> {idx}</p>
        <table style='width:100%; border-collapse: collapse; margin-top: 15px;'>
            <tr style='background-color: #34495e; color: white;'>
                <th style='padding: 12px; text-align: left; border: 1px solid #ddd;'>Experiential Dimension</th>
                <th style='padding: 12px; text-align: center; border: 1px solid #ddd;'>True Label</th>
                <th style='padding: 12px; text-align: center; border: 1px solid #ddd;'>Predicted Label</th>
                <th style='padding: 12px; text-align: center; border: 1px solid #ddd;'>Correct?</th>
            </tr>"""
    
    for i, label_name in enumerate(label_names):
        true_val = true_labels[i]
        pred_val = pred_labels[i]
        is_correct = true_val == pred_val
        
        if is_correct:
            bg_color = "#d5f4e6"
            status_color = "#27ae60"
            status_text = "Correct"
        else:
            bg_color = "#fadbd8"
            status_color = "#e74c3c"
            status_text = "Incorrect"
        
        true_text = "Yes" if true_val == 1 else "No"
        pred_text = "Yes" if pred_val == 1 else "No"
        
        html += f"""
            <tr style='background-color: {bg_color};'>
                <td style='padding: 10px; border: 1px solid #ddd; font-weight: bold;'>{label_name}</td>
                <td style='padding: 10px; text-align: center; border: 1px solid #ddd;'>{true_text}</td>
                <td style='padding: 10px; text-align: center; border: 1px solid #ddd;'>{pred_text}</td>
                <td style='padding: 10px; text-align: center; border: 1px solid #ddd; color: {status_color}; font-weight: bold;'>{status_text}</td>
            </tr>"""
    
    html += """
        </table>
    </div>
    """
    
    return HTML(html)

print("Prediction visualization functions defined")


## 3. Load Model Predictions


In [ ]:
# Load test predictions from training notebook
# Note: This assumes the training notebook has been run and predictions are saved
try:
    # Load predictions from the best model
    best_predictions = np.load('test_predictions.npy')
    
    # For demonstration, create dummy test labels if not available
    # In practice, these would be loaded from your test dataset
    dummy_test_labels = np.random.randint(0, 2, size=best_predictions.shape)
    
    print(f"Loaded predictions: {best_predictions.shape}")
    print(f"Created dummy test labels: {dummy_test_labels.shape}")
    
    # Analyze prediction quality
    analysis = analyze_prediction_quality(dummy_test_labels, best_predictions, label_names, "Best Model")
    
    print("Prediction Quality Analysis:")
    print(f"Perfect Correct: {len(analysis['perfect_correct'])} examples")
    print(f"Partially Correct: {len(analysis['partially_correct'])} examples")
    print(f"Completely Wrong: {len(analysis['completely_wrong'])} examples")
    print(f"False Positives: {len(analysis['false_positives'])} instances")
    print(f"False Negatives: {len(analysis['false_negatives'])} instances")
    print(f"Edge Cases: {len(analysis['edge_cases'])} examples")
    
except FileNotFoundError:
    print("test_predictions.npy not found. Please run the training notebook first.")
    print("Creating dummy data for demonstration...")
    
    # Create dummy data for demonstration
    n_samples = 100
    n_labels = len(label_names)
    best_predictions = np.random.randint(0, 2, size=(n_samples, n_labels))
    dummy_test_labels = np.random.randint(0, 2, size=(n_samples, n_labels))
    
    print(f"Created dummy predictions: {best_predictions.shape}")
    print(f"Created dummy test labels: {dummy_test_labels.shape}")


## 4. Prediction Examples Analysis


In [ ]:
# Display prediction examples
print("Perfect Correct Predictions:")
if len(analysis['perfect_correct']) > 0:
    for i, idx in enumerate(analysis['perfect_correct'][:2]):
        print(f"\n--- Example {i+1} ---")
        display(display_prediction_example(
            idx, dummy_test_labels, best_predictions, label_names, 
            "Best Model", "Perfect Correct"
        ))
else:
    print("No perfect correct examples found")

print("\nPartially Correct Predictions:")
if len(analysis['partially_correct']) > 0:
    for i, idx in enumerate(analysis['partially_correct'][:2]):
        print(f"\n--- Example {i+1} ---")
        display(display_prediction_example(
            idx, dummy_test_labels, best_predictions, label_names, 
            "Best Model", "Partially Correct"
        ))
else:
    print("No partially correct examples found")

print("\nCompletely Wrong Predictions:")
if len(analysis['completely_wrong']) > 0:
    for i, idx in enumerate(analysis['completely_wrong'][:2]):
        print(f"\n--- Example {i+1} ---")
        display(display_prediction_example(
            idx, dummy_test_labels, best_predictions, label_names, 
            "Best Model", "Completely Wrong"
        ))
else:
    print("No completely wrong examples found")


## 5. Error Pattern Analysis


In [ ]:
# Analyze error patterns
def analyze_error_patterns(analysis_results, model_name):
    """Analyze common error patterns for a model."""
    print(f"{model_name} Error Pattern Analysis:")
    print("-" * 40)
    
    # False Positive Analysis
    false_positives = analysis_results['false_positives']
    if false_positives:
        fp_by_label = {}
        for idx, label_idx, label_name in false_positives:
            if label_name not in fp_by_label:
                fp_by_label[label_name] = 0
            fp_by_label[label_name] += 1
        
        print("False Positives by Label:")
        for label_name, count in sorted(fp_by_label.items(), key=lambda x: x[1], reverse=True):
            print(f"   {label_name}: {count} instances")
    
    # False Negative Analysis
    false_negatives = analysis_results['false_negatives']
    if false_negatives:
        fn_by_label = {}
        for idx, label_idx, label_name in false_negatives:
            if label_name not in fn_by_label:
                fn_by_label[label_name] = 0
            fn_by_label[label_name] += 1
        
        print("\nFalse Negatives by Label:")
        for label_name, count in sorted(fn_by_label.items(), key=lambda x: x[1], reverse=True):
            print(f"   {label_name}: {count} instances")
    
    # Most problematic labels
    all_errors = false_positives + false_negatives
    if all_errors:
        error_by_label = {}
        for error in all_errors:
            label_name = error[2]
            if label_name not in error_by_label:
                error_by_label[label_name] = 0
            error_by_label[label_name] += 1
        
        print(f"\nMost Problematic Labels (Total Errors):")
        for label_name, count in sorted(error_by_label.items(), key=lambda x: x[1], reverse=True):
            print(f"   {label_name}: {count} total errors")

# Analyze error patterns
analyze_error_patterns(analysis, "Best Model")


## 6. Summary and Insights


In [ ]:
# Create summary statistics
def create_prediction_summary(analysis_results, model_name):
    """Create a summary of prediction quality."""
    total_examples = len(dummy_test_labels)
    
    perfect_correct = len(analysis_results['perfect_correct'])
    partially_correct = len(analysis_results['partially_correct'])
    completely_wrong = len(analysis_results['completely_wrong'])
    
    perfect_rate = (perfect_correct / total_examples) * 100
    partial_rate = (partially_correct / total_examples) * 100
    wrong_rate = (completely_wrong / total_examples) * 100
    
    print(f"{model_name} Prediction Quality Summary:")
    print(f"   Total Test Examples: {total_examples}")
    print(f"   Perfect Correct: {perfect_correct} ({perfect_rate:.1f}%)")
    print(f"   Partially Correct: {partially_correct} ({partial_rate:.1f}%)")
    print(f"   Completely Wrong: {completely_wrong} ({wrong_rate:.1f}%)")
    
    return {
        'model': model_name,
        'total_examples': total_examples,
        'perfect_correct': perfect_correct,
        'partially_correct': partially_correct,
        'completely_wrong': completely_wrong,
        'perfect_rate': perfect_rate,
        'partial_rate': partial_rate,
        'wrong_rate': wrong_rate
    }

# Create summary
summary = create_prediction_summary(analysis, "Best Model")

# Save analysis results
analysis_results = {
    'analysis': {
        'perfect_correct': analysis['perfect_correct'][:10],
        'partially_correct': analysis['partially_correct'][:10],
        'completely_wrong': analysis['completely_wrong'][:10],
        'edge_cases': analysis['edge_cases'][:5]
    },
    'summary': summary
}

with open('prediction_analysis_results.json', 'w') as f:
    json.dump(analysis_results, f, indent=2)

print("\nAnalysis results saved to prediction_analysis_results.json")
print("Model evaluation completed successfully")


## 7. Final Summary


In [ ]:
# Final evaluation summary
print("=" * 60)
print("MODEL EVALUATION SUMMARY")
print("=" * 60)

if 'analysis' in locals():
    print(f"\n📊 Prediction Quality Analysis:")
    print(f"  Perfect Correct: {len(analysis['perfect_correct'])} examples")
    print(f"  Partially Correct: {len(analysis['partially_correct'])} examples")
    print(f"  Completely Wrong: {len(analysis['completely_wrong'])} examples")
    print(f"  False Positives: {len(analysis['false_positives'])} instances")
    print(f"  False Negatives: {len(analysis['false_negatives'])} instances")

print(f"\n📁 Output Files Generated:")
print(f"  - prediction_analysis_results.json: Complete analysis results")
print(f"  - Interactive prediction examples displayed")

print(f"\n🔍 Analysis Features:")
print(f"  - Prediction quality categorization")
print(f"  - Error pattern analysis")
print(f"  - Interactive visualization")
print(f"  - Professional reporting")

print(f"\n✅ Model evaluation completed successfully!")
print("=" * 60)
